In [4]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Wed Nov  8 17:51:26 2017

@author: hph
"""

from astropy.io import fits
import numpy as np
from astropy.convolution import convolve_fft, Gaussian2DKernel
import math
import matplotlib.pyplot as plt
import astropy
print(astropy.__version__)


# Process residuals into a noise map
#residfile = fits.open('/Users/hph/cmzoom_catalog/catalog_5-28-19/CMZoom_residual_mosaic_without_sgra_pb_corrected.fits')
#resid = residfile[0].data[0][0][:][:]
residfile = fits.open('/Users/hph/cmzoom_catalog/catalog_acc/CMZoom_residual_pbcor.fits')

imagelength = residfile[0].data[:][:].size
resid = residfile[0].data[:][:]
resid_mask = resid
print(resid_mask.shape)
'''
for i in range(len(resid_mask[0][:])):
    for j in range(len(resid_mask[:][0])):
        if math.isnan(resid_mask[j][i]):
            resid_mask[j][i] = 0
        else:
            resid_mask[j][i] = 1
'''
#plt.imshow(resid_mask)
print( "starting first fft...")
#smresid = convolve_fft(np.nan_to_num(resid), Gaussian2DKernel(17), psf_pad=False, fft_pad=False, allow_huge=True)
smresid = convolve_fft(resid, Gaussian2DKernel(14), psf_pad=False, nan_treatment='interpolate',
                       fft_pad=False, allow_huge=True,preserve_nan=True,boundary='wrap')

#resid[np.isnan(resid)] = 10**9  #make the noise outside very high
noise_object = (resid-smresid)**2
print( "starting second fft...")
noise_sqr = convolve_fft(noise_object,  Gaussian2DKernel(14), psf_pad=False, nan_treatment='interpolate',
                         fft_pad=False, allow_huge=True,preserve_nan=True,boundary='wrap')
print("fft complete!")
noise =noise_sqr**0.5
noise[~np.isfinite(resid)] = np.nan
residfile[0].data = noise
residfile.writeto('/Users/hph/cmzoom_catalog/catalog_acc/CMZoom_noisemap_JySr_k14_final.fits',
                  overwrite=True)

print("Noise map written!")

'''
for filename in os.listdir('/Users/hph/current_fits/Residuals/residual_new_units/'):
    if filename.endswith('.fits'):
        # Process residuals into a noise map
        residfile = fits.open('/Users/hph/current_fits/Residuals/residual_new_units/'+filename)
        print filename
        #resid = residfile[0].data[0][0][:][:]
        resid = residfile[0].data[:][:]
        #smresid = convolve_fft(np.nan_to_num(resid), Gaussian2DKernel(30), allow_huge=True)
        smresid = convolve(np.nan_to_num(resid), Gaussian2DKernel(30))
        resid[np.isnan(resid)] = 10**11 # make the noise outside very high
        noise = convolve_fft((resid-smresid)**2,  Gaussian2DKernel(30), psf_pad=False, fft_pad=False)**0.5
        residfile[0].data = noise
        residfile.writeto('/Users/hph/current_fits/Residuals/noisemaps/'+filename+'_noisemap.fits', overwrite=True)
        print "Noise map written!"
    '''



print("complete")


3.2.1
(2800, 19000)
starting first fft...
starting second fft...
fft complete!
Noise map written!
complete
